# HW9: Beam Search Decoding - News Headline Generation

In this exercise, you are going to learn and implement decoding techniques for sequence generation. Usually, the sequence is generated word-by-word from a model. In each step, the model predicted the most likely word based on the predicted words in previous steps (this is called auto-regressive decoding).

As such, it is very important how you decide on what to predicted at each step, as it will be conditioned on to predicted all of the following steps. We will implement two of main decoding techniques introduced in the lecture: **Greedy Decoding** and **Beam Search Decoding**. Greedy Decoding immediately chooses the word with best score at each step, while Beam Search Decoding focuses on the sequence that give the best score overall.

To complete this exercise, you will need to complete the methods for decoding for a text generation model trained on [New York Times Comments and Headlines dataset](https://www.kaggle.com/aashita/nyt-comments). The model is trained to predict a headline for the news given seed text. You do not need to train any model model in this exercise as we provide both the pretrained model and dictionary.


## Download model and vocab and setup

In [1]:
!wget -O vocab.txt https://www.dropbox.com/s/ht12ua9vpkep6l8/hw9_vocab.txt?dl=0
!wget -O model.bin https://www.dropbox.com/s/okmri7cnd729rr5/hw9_model.bin?dl=0

--2023-03-28 13:45:51--  https://www.dropbox.com/s/ht12ua9vpkep6l8/hw9_vocab.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/ht12ua9vpkep6l8/hw9_vocab.txt [following]
--2023-03-28 13:45:51--  https://www.dropbox.com/s/raw/ht12ua9vpkep6l8/hw9_vocab.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccb9e6f332615e11bcbc3bd8b15.dl.dropboxusercontent.com/cd/0/inline/B5FTOY0PLEVm4W4C9cN0z6sdeD5K7_cGLtIqrLasi6I6eUR2S5nu_5DVHiphLinXec80LBixA1P6cWklxJF8Qb3WxAWH6gX0TyBQQZd38AWTLa1SnijNDq_HH1wEHMMaFtu4g9ZmCC55ptSFTuKrrYlj1-B6o-nXlqQ9Zf7rv16FYg/file# [following]
--2023-03-28 13:45:52--  https://uccb9e6f332615e11bcbc3bd8b15.dl.dropboxusercontent.com/cd/0/inline/B5FTOY0PLEVm4W4C9cN0z6sdeD5K7_cGLtIqrLasi6I6eUR2S5nu_5DVHiphLinXec80L

In [2]:
import torchtext
import torch
import torch.nn as nn
import numpy as np

In [3]:
class RNNmodel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout_rate):
                
        super().__init__()
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, 128, num_layers=2, 
                     batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, vocab_size)

    def forward(self, src):
        embedding = self.embedding(src)
        output,_ = self.rnn(embedding)         
        output = self.dropout(output) 
        prediction = self.fc2(output)
        return prediction

In [4]:
with open("vocab.txt") as f:
  vocab_file = f.readlines()
embedding_dim = 64             
dropout_rate = 0.2              

model = RNNmodel(len(vocab_file), embedding_dim, dropout_rate)
model.load_state_dict(torch.load("model.bin",map_location='cpu'))
model.eval()

RNNmodel(
  (embedding): Embedding(10054, 64)
  (rnn): LSTM(64, 128, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=128, out_features=10054, bias=True)
)

In [5]:
vocab = torchtext.vocab.vocab({})
for v in vocab_file: vocab.append_token(v.strip()) 

## 1. TODO: Greedy decode
Normally, in sequence generation task, the model will continue generating tokens until an end-of-sequence symbol appear or the maximum length is reached. For this task:
- The end-of-sequence symbol is "< eos >" and its index is 2
- Use the maximum generation length of 15

In [102]:
eos_token = '<eos>'
max_gen_length = 15

In [132]:
def greedy_decode(seed_text, vocab, max_gen_length=15):
    """Greedy decodes with seed text.

        Args:
        seed_text: The seed string to be used as initial input to the model.
        vocab: The vocab for converting word to index and back.

        Your code should do the followings:
          1. Convert current_text to sequences of indices 
          2. Predict the next token using the model and choose the token with the highest score as output
          3. Append the predicted index to current_text
          4. Loop until completion
          5. Return text prediction and a list of probabilities of each step

        You do not need to stop early when end-of-sequence token is generated and can continue decoding
        until max_gen_length is reached. We can filter the eos token out later.
    """
    word_to_idx = vocab.get_stoi()
    idx_to_word = vocab.get_itos()
    output = [seed_text]
    probs = []
    output_tensor = torch.LongTensor([word_to_idx[seed_text]])
    with torch.no_grad():
      while output_tensor.shape[0] <= max_gen_length:
        pred = torch.softmax(model(output_tensor), dim=1)
        pred_id, pred_prob = pred[-1].max(dim=0).indices, pred[-1].max(dim=0).values
        output += [idx_to_word[pred_id]]
        probs += [pred_prob.item()]
        output_tensor = torch.LongTensor([word_to_idx[t] for t in output])
    return output, probs

In [133]:
def clean_output(text, eos_token):
    """Drop eos_token and every words that follow"""
    return ' '.join(text[:text.index(eos_token)]) if eos_token in text else ' '.join(text)

In [134]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
for seed in sample_seeds:
    output, probs = greedy_decode(seed, vocab)
    print(clean_output(output, eos_token))
    

to encourage creativity in the new york bill
america s lethal export
people to balloon to make a criminal with a dog with a callous rival
next phenom english clubs 2 call another deal in the same arrivals
picture perfect chapter a spot of view of banning care
on the catwalk in saudi arabia


Your output should be:

*   to encourage creativity in the new york bill 
*   america s lethal export
*   people to balloon to make a criminal with a dog with a callous rival 
*   next phenom english clubs 2 call another deal in the same arrivals
*   picture perfect chapter a spot of view of banning care  
*   on the catwalk in saudi arabia
 

 

 


## 2. TODO: Beam search decode

Another well-known decoding method is beam search decoding that focuses more on the overall sequence score.

Instead of greedily choosing the token with the highest score for each step, beam search decoding expands all possible next tokens and keeps the __k__ most likely sequence at each step, where __k__ is a user-specified beam size. A sequence score is also calculated according user-specified cal_score() function.
The beam with the highest score after the decoding process is done will be the output.

There are a few things that you need to know before implementing a beam search decoder:
- When the eos token is produced, you can stop expanding that beam
- However, the ended beams must be sorted together with active beams
- The decoding ends when every beams are either ended or reached the maximum length, but for this task, you can continue decoding until the max_gen_len is reached
- We usually work with probability in log scale to avoid numerical underflow. You should use np.log(score) before any calculation
- **As probabilities for some classes will be very small, you must add a very small value to the score before taking log e.g np.log(prob + 0.00000001)**

#### Sequence Score
The naive way to calculate the sequence score is to __multiply every token scores__ together. However, doing so will make the decoder prefer shorter sequence as you multiply the sequence score with a value between \[0,1\] for every tokens in the sequence. Thus, we usually normalize the sequence score with its length by calculating its __geometric mean__ instead.

**You should do this in log scale**

In [135]:
def cal_score(score_list, length, normalized=False): #cal score for each beam from a list of probs
    eps = 0.00000001
    if normalized:
        score = np.log(np.array(score_list) + eps).sum()/length
    else:
        score = np.log(np.array(score_list) + eps).sum()
    return score

In [136]:
def beam_search_decode(seed_text, max_gen_len, vocab, beam_size=5, normalized=False):
    """We will do beam search decoding using seed text in this function.

    Output:
    beams: A list of top k beams after the decoding ended, each beam is a list of 
      [seed_text, list of scores, length]

    Your code should do the followings:
    1.Loop until max_gen_len is reached.
    2.During each step, loop through each beam and use it to predict the next word.
      If a beam is already ended, continues without expanding.
    3.Sort all hypotheses according to cal_score().
    4.Keep top k hypotheses to be used at the next step.
    """
    # For each beam we will store (generated text, list of scores, and current length, is_finished)
    # Add initial beam
    word_to_idx = vocab.get_stoi()
    idx_to_word = vocab.get_itos()
    beams = [[[seed_text], [], 0, False]]
    for _ in range(max_gen_len):
      candidate_beams = []
      for beam in beams:
        texts, scores, curr_len, finish_flag = beam
        if finish_flag:
          candidate_beams.append(beam)
          continue
        texts_tensor = torch.LongTensor([word_to_idx[t] for t in texts])
        # print(texts_tensor)
        with torch.no_grad():
          pred = torch.softmax(model(texts_tensor), dim=1)
          candidate_idx = pred[-1].argsort(dim=0, descending=True)[:beam_size]
          candidate_probs = pred[-1][candidate_idx]
          # print(candidate_idx)
          # print([idx_to_word[id] for id in candidate_idx])
          # print(candidate_probs)
        for c_idx, c_prob in zip(candidate_idx.tolist(), candidate_probs.tolist()):
          # print(c_idx, c_prob)
          if idx_to_word[c_idx] == '<eos>':
            next_finish_flag = True
            candidate_beams.append([
              texts,
              scores + [c_prob],
              curr_len + 1,
              next_finish_flag 
              ])
          else:
            next_finish_flag = False
            candidate_beams.append([
              texts + [idx_to_word[c_idx]],
              scores + [c_prob],
              curr_len + 1,
              next_finish_flag 
              ])
        candidate_beams.sort(reverse=True, key=lambda beam: cal_score(beam[1], beam[2], normalized=normalized))
        candidate_beams = candidate_beams[:beam_size]
        # print(candidate_beams)
        beams = candidate_beams[:]
        # for b in beams:
        #   print(" ".join(b[0]), cal_score(b[1], b[2], normalized))
        # print()
        # print([(" ".join(cb[0]), cal_score(cb[1], cb[2])) for cb in candidate_beams])
      # print()

    return beams

## 3. Generate!
Generate 6 sentences based on the given seed texts.

Decode with the provided seed texts with beam_size 5. Compare the results between greedy, normalized, and unnormalized decoding.

Print the result using greedy decoding and top 2 results each using unnormalized and normalized decoing for each seed text.

Also, print scores of each candidate according to cal_score(). Use normalization for greedy decoding.

In [137]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
max_gen_length = 10
for seed in sample_seeds:
    output, probs = greedy_decode(seed, vocab)
    beams = beam_search_decode(seed, max_gen_length, vocab, beam_size=5, normalized=False)
    beams_norm = beam_search_decode(seed, max_gen_length, vocab, beam_size=5, normalized=True)
    
    print('-Greedy-')
    print(clean_output(output, eos_token))
    print('-Unnormalized-')
    print(' '.join(beams[0][0]), round(np.e**cal_score(beams[0][1], beams[0][2], normalized=False), 2) )
    print(' '.join(beams[1][0]), round(np.e**cal_score(beams[1][1], beams[1][2], normalized=False), 2) )
    print('-Normalized-')
    print(' '.join(beams_norm[0][0]), round(np.e**cal_score(beams_norm[0][1], beams_norm[0][2], normalized=True), 2) )
    print(' '.join(beams_norm[1][0]), round(np.e**cal_score(beams_norm[1][1], beams_norm[1][2], normalized=True), 2) )
    print()
    


-Greedy-
to encourage creativity in the new york bill
-Unnormalized-
to consult exploring recipes for new jersey 0.0
to consult exploring recipes up the pacific northwest 0.0
-Normalized-
to consult exploring recipes up the pacific northwest 0.22
to consult exploring recipes up the least of the week 0.19

-Greedy-
america s lethal export
-Unnormalized-
america s lethal export 0.01
america s desert aisles 0.01
-Normalized-
america s lethal export 0.35
america s desert aisles 0.3

-Greedy-
people to balloon to make a criminal with a dog with a callous rival
-Unnormalized-
people to balloon for a criminal 0.0
people to balloon for a criminal with trump 0.0
-Normalized-
people to balloon for a criminal with a second fiddle 0.16
people to balloon for a criminal with trump 0.16

-Greedy-
next phenom english clubs 2 call another deal in the same arrivals
-Unnormalized-
next s blist revue 0.0
next phenom english clubs 1 a chance to be back 0.0
-Normalized-
next s blist revue 0.23
next phenom e

Your output should be:


```
-Greedy-
to encourage creativity in the new york bill  0.12
-Unnormalized-
To Consult Exploring Recipes For New Jersey 0.00
To Consult Exploring Recipes Up The Pacific Northwest 0.00
-Normalized-
To Consult Exploring Recipes Up The Pacific Northwest 0.17
To Consult Exploring Recipes Up The Least Of The Week 0.16

-Greedy-
america s lethal export  0.35
-Unnormalized-
America S Lethal Export 0.02
America S Desert Aisles 0.01
-Normalized-
America S Lethal Export 0.25
America S Desert Aisles 0.20

-Greedy-
people to balloon to make a criminal with a dog with a callous rival  0.16
-Unnormalized-
People To Balloon For A Criminal 0.00
People To Balloon For A Criminal With Trump 0.00
-Normalized-
People To Balloon For A Criminal With A Second Fiddle 0.13
People To Balloon For A Criminal With Trump 0.13

-Greedy-
next phenom english clubs 2 call another deal in the same arrivals  0.15
-Unnormalized-
Next S Blist Revue 0.00
Next Phenom English Clubs 1 A Chance To Be Back 0.00
-Normalized-
Next S Blist Revue 0.14
Next Phenom English Clubs 1 A Chance To Be Back 0.14

-Greedy-
picture perfect chapter a spot of view of banning care  0.09
-Unnormalized-
Picture Perfect Use Coffee 0.00
Picture Korean A Bonanza Of Pancakes 0.00
-Normalized-
Picture Korean A Bonanza Of Contemplation Times Of Trump S Son 0.12
Picture Korean A Bonanza Of Pancakes 0.07

-Greedy-
on the catwalk in saudi arabia  0.25
-Unnormalized-
On The Billboard Chart 0.00
On The Catwalk In Saudi Arabia 0.00
-Normalized-
On The Whole30 Diet Vowing To Eat Smarter Carbs To Be 0.27
On The Whole30 Diet Vowing To Eat Smarter Carbs For Because 0.26

```



# Answer Questions in MyCourseVille!

Use the seed word "usa" to answer questions in MCV.

In [138]:
sample_seeds = ["usa"]
max_gen_length = 10
for seed in sample_seeds:
    output, probs = greedy_decode(seed, vocab)
    beams = beam_search_decode(seed, max_gen_length, vocab, beam_size=5, normalized=False)
    beams_norm = beam_search_decode(seed, max_gen_length, vocab, beam_size=5, normalized=True)
    
    print('-Greedy-')
    print(clean_output(output, eos_token))
    print('-Unnormalized-')
    print(' '.join(beams[0][0]), round(np.e**cal_score(beams[0][1], beams[0][2], normalized=False), 2) )
    print(' '.join(beams[1][0]), round(np.e**cal_score(beams[1][1], beams[1][2], normalized=False), 2) )
    print('-Normalized-')
    print(' '.join(beams_norm[0][0]), round(np.e**cal_score(beams_norm[0][1], beams_norm[0][2], normalized=True), 2) )
    print(' '.join(beams_norm[1][0]), round(np.e**cal_score(beams_norm[1][1], beams_norm[1][2], normalized=True), 2) )
    print()
    


-Greedy-
usa s duty to investigate
-Unnormalized-
usa s duty to investigate 0.0
usa s bleak season 3 episode 4 recap chicken 0.0
-Normalized-
usa s bleak season 1 episode 2 darkness descends 0.23
usa s bleak season 3 episode 4 recap a seduction 0.21

